In [13]:
from skimage import feature
from skimage.util import img_as_ubyte
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [14]:
image_raw = cv2.imread('02_80-100s__crop256px_5_5.bmp')

In [17]:
type(image_raw)

NoneType

In [15]:
image_gb = cv2.GaussianBlur(image_raw,(5,5), cv2.BORDER_DEFAULT)

In [16]:
plt.imshow(image_raw)
plt.show()

TypeError: Image data of dtype object cannot be converted to float